In [1]:
import os
import re
import cgi
import requests
import mimetypes
import xlrd
import xlsxwriter
import time
import requests
import csv
import json
import os.path
import chardet

# input model

In [2]:
access_rawdata = r''
def getRows():
    with open(access_rawdata, encoding = 'utf-8') as csvfile:
        spamreader = csv.reader(csvfile, delimiter = ',')
        next(spamreader)
        models = list()
        for row in spamreader:
            model = dict()
            model['nid'] = row[0]
            model['title'] = row[1]
            model['subOrg'] = row[2]
            model['dataset_link'] = row[3]
            model['field_description'] = row[4]
            model['resource_link'] = row[5]
            model['resource_Writeformat'] = row[6]
            model['resource_WriteCharset'] = row[7]
            model['linkcode'] = None
            model['true_format'] = None
            model['true_Charset'] = None
            model['columnAlike'] = None
            models.append(model)
        return (models)

In [3]:
data_list = getRows()

# machine-readable(structural)

In [4]:
def get_machine_readable():
    machine_readable = [
        'csv',
        'CSV',
        'json',
        'xml',
        'vnd.google-earth.kml+xml',
        'xlsx',
        'kmz',
        'xls',
        'ods',
        'ODS',
        'XML',
        'vnd.ms-excel',
        'vnd.ms-excel.12'
        
    ]
    return machine_readable

# 連線測試:查看http response status code & file format

In [5]:
def linkingTest(url):
    try:
        response = requests.get(url, timeout = 3000)
        #get the http response status code
        status = response.status_code
        linkcode = response
        try:
            if 'Content-Type' in response.headers:
                typeparams, charset = cgi.parse_header(response.headers.get('Content-Type', ''))
                content_type = (typeparams.split('/')[1])
                if content_type == 'octet-stream':
                    try:
                        _, filenameparams = cgi.parse_header(response.headers.get('Content-Disposition',''))
                        filename = filenameparams['filename']
                        extension = os.path.splitext(filename)[1].strip('.')
                        true_format = extension
                    except:
                        m = re.search(r'(\.\w*$)', url)
                        if m:
                            true_format = m.group(1).strip('.')
                        else:
                            true_format = 'unknow'
                else:
                    true_format = content_type
            else:
                true_format = "NoFileFounded"
        except:
            true_format = 'unknow'
    except:
        linkcode = 'Error'
        true_format = 'Error'
    return linkcode, true_format

# charset detect

In [6]:
def charsetTest(url):
    try:
        with requests.Session() as s:
            download = s.get(url, timeout =10)
            content = download.content
            result = chardet.detect(content)
            charset = result['encoding']
    except:
        charset = 'None'
    return charset
    

# field description clean

In [7]:
def fieldDescriptionClean(column):
    description = column.replace('\n','').replace('\t','').strip()
    description_list = (re.split('、|,|，',description))
    
    try:
        for n, col in enumerate(description_list):
            if col.find('(') and col.endswith(')'):
                m = re.match(r'(.*)\(.*\)',col)
                description_list[n] = m.group(1)
    except:
        pass
    
    return description_list

# csv column matcher

In [8]:
def csvColMatching(url,charset,writed_column):
    with requests.Session() as s:
        try:
            download = s.get(url)
            decoded_content = download.content.decode(charset)
            cr = csv.reader(decoded_content.splitlines(), delimiter = ',')
            content_list = list(cr)
            headingCol = content_list[1]
            if '\ufeff' in headingCol[0]:
                headingCol[0] = headingCol[0].replace('\ufeff', '')
            if set(writed_column) == set(headingCol):
                Col_matching = 'True'
            else:
                Col_matching = 'False'

        except Exception as e:
            Col_matching = str(e)
    return Col_matching

# xml column matcher

In [9]:
def xmlColMatching(url, charset, writed_column):
    try:
        XML = requests.get(url, timeout = 10).content.decode(charset)
        root = ET.fromstring(XML)
        tag_list = list()
        for descendant in root.findall(".//*"):
            if descendant.tag not in tag_list:
                tag_list.append(descendant.tag)
        for n,tag in enumerate(tag_list):
            if tag.startswith('{') and tag.find('}'):
                m = re.match(r'({.*})(.*)',tag)
                tag_list[n] = m.group(2)
        if len(list(set(writed_column).intersection(set(tag_list)))) == len(writed_column):
            Col_matching = 'True'
        else:
            Col_matching = 'False'
    except Exception as e:
        Col_matching = str(e)
    return Col_matching
            

# json column matcher

In [10]:
def jsonColMatching(url,charset,writed_column):
    try:
        with requests.Session() as s:
            download = s.get(url, timeout = 10)
            decoded_content = download.content.decode(charset)
            json_obj = json.loads(decoded_content)
            json_key = list()
            for i in json_obj:
                for key in i.keys():
                    if key not in json_key:
                        json_key.append(key)
            if set(writed_column) == set(json_key):
                Col_matching = 'True'
            else:
                Col_matching = 'Flase'
    except Exception as e:
            Col_matching = str(e)
    return Col_matching

# xls and xlsx column matcher

In [11]:
def excelColMatching(url, writed_column):
    try:
        with requests.Session() as s:
            download = s.get(url, timeout = 10)
            decoded_content = download.content
            wb = xlrd.open_workbook(file_contents = decoded_content, encoding_override = 'cp950')
            table = wb.sheets()[0]
            nrows = table.nrows
            xls_heading = table.row_values(0)
        if set (xls_heading) == set (writed_column):
            Col_matching = 'True'
        else:
            Col_matching = 'False'
    except Exception as e:
        Col_matching = str(e)
    return Col_matching

# Run

In [12]:
import sys
machine_readable = get_machine_readable()
for data in data_list:
    url = data['resource_link']
    write_Charset = data['resource_WriteCharset']
    link_test = linkingTest(url)
    data['linkcode'] = link_test[0]
    data['true_format'] = link_test[1]
    if data['true_format'] in ('', 'plain','unknow'):
        data['true_format'] = data['resource_Writeformat']
    
    
    if data['true_format'] in machine_readable:
        charset = charsetTest(url)
        data['true_Charset'] = charset
        description_list = fieldDescriptionClean(data['field_description'])
        
        if data['true_format'] == 'CSV' or 'csv':
            ColMatching = csvColMatching(url, charset, description_list)
            data['columnAlike'] = ColMatching

        elif data['true_format'] == 'xml':
            ColMatching = xmlColMatching(url, charset, description_list)
            data['columnAlike'] = ColMatching
        elif data['true_format'] == 'json':
            ColMatching = jsonColMatching(url, description_list)
            data['columnAlike'] = ColMatching

        elif data['true_format'] == 'xls' or data['true_format'] == 'xlsx':
            ColMatching = excelColMatching(url, description_list)
            data['columnAlike'] = ColMatching
            
        

# Excel Writing

In [13]:
print("CREATE EXCEL...")
workbook = xlsxwriter.Workbook('Tourism_result_105_09_19_1.xls', {'strings_to_urls': False})
worksheet = workbook.add_worksheet(u'評鑑範例表')
row = 0
col = 0
# print title
worksheet.write(row, col, u"資料集Id")
worksheet.write(row, col + 1, u"資料集名稱")
worksheet.write(row, col + 2, u"資料所屬機關")
worksheet.write(row, col + 3, u"資料集連結")
worksheet.write(row, col + 4, u"主要欄位說明")
worksheet.write(row, col + 5, u"資料資源下載連結")
worksheet.write(row, col + 6, u"詮釋資料檔案格式")
worksheet.write(row, col + 7, u"詮釋資料文字編碼")
worksheet.write(row, col + 8, u"連線狀態")
worksheet.write(row, col + 9, u"資料資源檔案格式")
worksheet.write(row, col + 10, u"資料資源文字編碼")
worksheet.write(row, col + 11, u"欄位是否相符")

row += 1
for i in data_list:
    nid = i['nid']
    title = i['title']
    subOrg = i['subOrg']
    dataset_link = i['dataset_link']
    field_description = i['field_description']
    resource_link = i['resource_link']
    resource_Writeformat = i['resource_Writeformat']
    resource_WriteCharset = i['resource_WriteCharset']
    linkStatus = i['linkcode']
    TrueFormat = i['true_format']
    TrueCharset = i['true_Charset']
    columnAlike = i['columnAlike']
        
    worksheet.write(row, col, str(nid))
    worksheet.write(row, col + 1, title)
    worksheet.write(row, col + 2, subOrg)
    worksheet.write(row, col + 3, dataset_link)
    worksheet.write(row, col + 4, field_description)
    worksheet.write(row, col + 5, resource_link)
    worksheet.write(row, col + 6, resource_Writeformat)
    worksheet.write(row, col + 7, resource_WriteCharset)
    worksheet.write(row, col + 8, str(linkStatus))
    worksheet.write(row, col + 9, TrueFormat)
    worksheet.write(row, col + 10, str(TrueCharset))
    worksheet.write(row, col + 11, columnAlike)
    row += 1
time.sleep(3)
workbook.close()

CREATE EXCEL...
